In [1]:
from textblob import TextBlob

In [2]:
blob = TextBlob("I really enjoy coding with Python")

# 감정 분석
주관성은 개인적인 의견을 말합니다. 주관성은 (0,1) 범위에있는 부동 소수점입니다. 주관성의 값이 0.5 이상이면 문장이 객관적보다 주관적이며 그 반대의 경우도 마찬가지입니다. 문장의 주관성은 0.48이므로 주관적보다는 객관적입니다.

In [3]:
result = blob.sentiment

In [4]:
# subjectivity 주관성
# polarity 극성 +면 긍정 -면 부정
print(result)

Sentiment(polarity=0.4, subjectivity=0.5)


In [5]:
print(result.polarity)

0.4


In [6]:
blob = TextBlob("I kinda of think think taht you are very bad at Python")

In [7]:
print(blob.sentiment)

Sentiment(polarity=-0.9099999999999998, subjectivity=0.8666666666666667)


In [9]:
blob = TextBlob("TextBlob has 7700 stars on GitHub.")

In [10]:
print(blob.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


In [19]:
blob = TextBlob("Emotional Korean language... Emotional Korea where people who give rational opinions are treated strangely...")

In [20]:
print(blob.sentiment)

Sentiment(polarity=-0.016666666666666666, subjectivity=0.48333333333333334)


# 감성분석 
데이터를 학습한 모델로 판단해보기

In [21]:
# 데이터 셋 다운
!pip install datasets


  Using cached pyarrow-5.0.0-cp38-cp38-win_amd64.whl (14.5 MB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)


In [23]:
from datasets import load_dataset

In [24]:
# 학습용 데이터 다운
train_data = load_dataset("glue", "sst2", split="train[:1999]")

Downloading: 28.8kB [00:00, 14.4MB/s]                   
Downloading: 28.7kB [00:00, 14.3MB/s]                   


Downloading: 100%|██████████| 7.44M/7.44M [00:01<00:00, 5.37MB/s]
                                

Dataset glue downloaded and prepared to C:\Users\user\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [41]:
type(train_data), len(train_data)

(datasets.arrow_dataset.Dataset, 1999)

In [42]:
train_data[:10]

{'sentence': ['hide new secretions from the parental units ',
  'contains no wit , only labored gags ',
  'that loves its characters and communicates something rather beautiful about human nature ',
  'remains utterly satisfied to remain the same throughout ',
  'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
  "that 's far too tragic to merit such superficial treatment ",
  'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
  'of saucy ',
  "a depressed fifteen-year-old 's suicidal poetry ",
  "are more deeply thought through than in most ` right-thinking ' films "],
 'label': [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [25]:
# 평가(검증)용 데이터 다운
eval_data = load_dataset("glue", "sst2", split="validation[:]")

Reusing dataset glue (C:\Users\user\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [56]:
type(eval_data), len(eval_data)

(datasets.arrow_dataset.Dataset, 872)

In [57]:
eval_data[:10]

{'sentence': ["it 's a charming and often affecting journey . ",
  'unflinchingly bleak and desperate ',
  'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
  "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
  "it 's slow -- very , very slow . ",
  'although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . ',
  'a sometimes tedious film . ',
  "or doing last year 's taxes with your ex-wife . ",
  "you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . ",
  "in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter turkey . "],
 'label': [1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [58]:
# 문장과 라벨 출력
for case in eval_data:
    print(case["sentence"])
    print(case["label"])

it 's a charming and often affecting journey . 
1
unflinchingly bleak and desperate 
0
allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
1
the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
1
it 's slow -- very , very slow . 
0
although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . 
1
a sometimes tedious film . 
0
or doing last year 's taxes with your ex-wife . 
0
you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . 
1
in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter turkey . 
0
the mesmerizing performances of the leads keep the film grounded and keep the audience riveted . 
1
it takes a strange kind of laziness to waste the talents of robert forster , anne meara ,

In [59]:
textblob_train_data =[]

In [60]:
#학습용 데이터로 라벨링 해주기
for case in train_data:
    label = "positive" if case["label"] else "negative"
    case_tuple = (case["sentence"], label)
    textblob_train_data.append(case_tuple)

In [61]:
for case in textblob_train_data:
    print(case)

('hide new secretions from the parental units ', 'negative')
('contains no wit , only labored gags ', 'negative')
('that loves its characters and communicates something rather beautiful about human nature ', 'positive')
('remains utterly satisfied to remain the same throughout ', 'negative')
('on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'negative')
("that 's far too tragic to merit such superficial treatment ", 'negative')
('demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ', 'positive')
('of saucy ', 'positive')
("a depressed fifteen-year-old 's suicidal poetry ", 'negative')
("are more deeply thought through than in most ` right-thinking ' films ", 'positive')
('goes to absurd lengths ', 'negative')
("for those moviegoers who complain that ` they do n't make movies like they used to anymore ", 'negative')
("the part where nothing 's happening , ", 'negat

In [62]:
# 이번엔 평가용 데이터로 튜플 만들기
textblob_eval_data =[]
for case in eval_data:
    label = "positive" if case["label"] else "negative"
    case_tuple = (case["sentence"], label)
    textblob_eval_data.append(case_tuple)
for case in textblob_eval_data:
    print(case)

("it 's a charming and often affecting journey . ", 'positive')
('unflinchingly bleak and desperate ', 'negative')
('allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ', 'positive')
("the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ", 'positive')
("it 's slow -- very , very slow . ", 'negative')
('although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . ', 'positive')
('a sometimes tedious film . ', 'negative')
("or doing last year 's taxes with your ex-wife . ", 'negative')
("you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . ", 'positive')
("in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter turkey . ", 'negative')
('the mesmerizing performances of the leads keep 

In [63]:
# 나이브 베이즈 모델을 사용할 예정
from textblob.classifiers import NaiveBayesClassifier

In [64]:
# 모델에 데이터 집어넣기
classifier = NaiveBayesClassifier(textblob_train_data)

In [65]:
result = classifier.accuracy(textblob_eval_data)

In [66]:
# 모델의 성능은 0.7정확도
print(result)

0.7236238532110092


# 긍정 부정 분류하기

In [69]:
result = classifier.classify("This was a great movie!")
print(result)

positive


In [70]:
result = classifier.classify("This was a bad movie!")
print(result)

negative


In [71]:
probs_result = classifier.prob_classify("This was a great movie!")
print(probs_result)

<ProbDist with 2 samples>


# 긍정 확률 부정확률 확인 ("This was a great movie!")일때

In [74]:
pos_result = probs_result.prob("positive")
print(pos_result)

0.9208174887068612


In [75]:
pos_result = probs_result.prob("negative")
print(pos_result)

0.07918251129313693


In [76]:
probs_result = classifier.prob_classify("This was a bad movie!")
print(probs_result)

<ProbDist with 2 samples>


In [77]:
pos_result = probs_result.prob("positive")
print(pos_result)

0.05856517283252036
